In [18]:
import numpy as np
import h5py as h5
import osr
import gdal

In [2]:
cd

/home/s1949330


In [3]:
cd Documents/initial_data

/home/s1949330/Documents/initial_data


In [4]:
## Reading GEDI

class readGEDI():
    def __init__(self,fileList):
        gedi_f = h5.File('GEDI02_B_2020_04_30_2020124194427_O07882_01_T02809_02_003_01_V002.h5', 'r')
        beamList = ['BEAM0000','BEAM0001','BEAM0010','BEAM0011','BEAM0101','BEAM0110','BEAM1000','BEAM1011']       
                
        self.x = np.empty((), dtype = float)
        self.y = np.empty((), dtype = float)
        self.sens = np.empty((), dtype = float)
        self.cover = np.empty((), dtype = float)
        
        for beam in beamList:
            self.x = np.append(self.x, np.array(gedi_f[beam]['geolocation/lon_lowestmode']))
            self.y = np.append(self.y, np.array(gedi_f[beam]['geolocation/lat_lowestmode']))
            self.sens = np.append(self.sens, np.array(gedi_f[beam]['sensitivity']))
            self.cover = np.append(self.cover, np.array(gedi_f[beam]['cover']))
        

In [5]:
gedi = readGEDI('fileList')
print(gedi.x)
print(gedi.y)
print(gedi.sens)
print(gedi.cover)

[-0.1        -0.17432582 -0.17403249 ... 43.89469689 43.89499315
 43.8952898 ]
[  0.         -43.36818741 -43.36804818 ...  -0.07988222  -0.07946151
  -0.07904034]
[0.         4.61821699 4.1790204  ... 0.76948482 0.78958929 0.72204185]
[ 2.00000000e+00 -9.99900000e+03 -9.99900000e+03 ...  1.97091792e-02
  5.04121557e-03  1.79270748e-02]


In [8]:
cd

/home/s1949330


In [9]:
cd Documents/initial_data

/home/s1949330/Documents/initial_data


In [10]:
ls

GEDI02_B_2020_04_30_2020124194427_O07882_01_T02809_02_003_01_V002.h5
T36LWK_A025400_20200503T075551_B01.jp2
T36LWK_A025400_20200503T075551_B02.jp2
T36LWK_A025400_20200503T075551_B03.jp2
T36LWK_A025400_20200503T075551_B04.jp2
T36LWK_A025400_20200503T075551_B05.jp2
T36LWK_A025400_20200503T075551_B06.jp2
T36LWK_A025400_20200503T075551_B07.jp2
T36LWK_A025400_20200503T075551_B08.jp2
T36LWK_A025400_20200503T075551_B09.jp2
T36LWK_A025400_20200503T075551_B11.jp2
T36LWK_A025400_20200503T075551_B12.jp2
T36LWK_A025400_20200503T075551_B8A.jp2


In [11]:
bandList_all = ['T36LWK_A025400_20200503T075551_B01.jp2',
                'T36LWK_A025400_20200503T075551_B02.jp2',
                'T36LWK_A025400_20200503T075551_B03.jp2',
                'T36LWK_A025400_20200503T075551_B04.jp2',
                'T36LWK_A025400_20200503T075551_B05.jp2',
                'T36LWK_A025400_20200503T075551_B06.jp2',
                'T36LWK_A025400_20200503T075551_B07.jp2',
                'T36LWK_A025400_20200503T075551_B08.jp2',
                'T36LWK_A025400_20200503T075551_B09.jp2',
                'T36LWK_A025400_20200503T075551_B11.jp2',
                'T36LWK_A025400_20200503T075551_B12.jp2',
                'T36LWK_A025400_20200503T075551_B8A.jp2']

bandList_10 = ['T36LWK_A025400_20200503T075551_B02.jp2',
               'T36LWK_A025400_20200503T075551_B03.jp2',
               'T36LWK_A025400_20200503T075551_B04.jp2',
               'T36LWK_A025400_20200503T075551_B08.jp2']

bandList_20 = ['T36LWK_A025400_20200503T075551_B05.jp2',
               'T36LWK_A025400_20200503T075551_B06.jp2',
               'T36LWK_A025400_20200503T075551_B07.jp2',
               'T36LWK_A025400_20200503T075551_B8A.jp2']

In [6]:
# Coarsen Sentinel-2 bands

# def writeTiff(data, x, y, res, filename = 'gedi_intersect.tif', epsg=4326)


In [ ]:
### Reading Sentinel-2

from osgeo import gdal
from math import floor
import numpy as np
import osr

from glob import glob

dir='/home/s1949330/Documents/initial_data'



sentList=glob(dir+'/*.jp2')


class readSentinel():
    def __init__(self,fileName):
        ds = gdal.Open(fileName)
        proj = osr.SpatialReference(wkt = ds.GetProjection())
        self.epsg = int(proj.GetAttrValue('AUTHORITY', 1))
        
        self.nX = ds.RasterXSize
        self.nY = ds.RasterYSize
        
        transform_ds = ds.GetGeoTransform()
        self.xOrigin = transform_ds[0]
        self.yOrigin = transform_ds[3]
        self.pixelWidth = transform_ds[1]
        self.pixelHeight = transform_ds[5]
        
        self.data = ds.GetRasterBand(1).ReadAsArray(0, 0, self.nX, self.nY)
        
        
    def coarsen(self,res):
        '''Coarsen resolutin of data layer'''
        print('Coarsening')
        
        # dimensions of output image
        newX=floor(self.nX*self.pixelWidth/res)
        newY=floor(self.nY*self.pixelHeight/(-1*res))
        
        # allocate space
        newData=np.zeros((newX,newY),dtype=float)
        contN=np.zeros((newX,newY),dtype=int)
        
        # loop iover inpuit image and take rolling sum
        for i in range(0,self.nX):
            print('Column',i,'of',self.nX)
            for j in range(0,self.nY):
                xInd=floor(i*self.pixelWidth/res)
                yInd=floor(j*self.pixelHeight/res)
                newData[xInd,yInd]+=self.data[i,j]
                contN[xInd,yInd]+=1
                
        # normalise means
        newData[contN>0]=newData[contN>0]/contN[contN>0]
        newData[contN==0]=-999
        
        self.pixelWidth=res
        self.pixelHeight=-1*res
        self.data=newData
        
        
i = 0
s2=np.empty((12),dtype=readSentinel)
useList=np.full((12),True,dtype=bool)

print(s2.shape)

for filename in sentList:
    print(i,filename)

    s2[i] = readSentinel(filename)
    
    if s2[i].pixelWidth==60:
        useList[i]=False
    elif s2[i].pixelWidth==10:
        s2[i].coarsen(20)
    
    i=i+1
    

(12,)
0 /home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B07.jp2
1 /home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B08.jp2
Coarsening
Column 0 of 10980
Column 1 of 10980
Column 2 of 10980
Column 3 of 10980
Column 4 of 10980
Column 5 of 10980
Column 6 of 10980
Column 7 of 10980
Column 8 of 10980
Column 9 of 10980
Column 10 of 10980
Column 11 of 10980
Column 12 of 10980
Column 13 of 10980
Column 14 of 10980
Column 15 of 10980
Column 16 of 10980
Column 17 of 10980
Column 18 of 10980
Column 19 of 10980
Column 20 of 10980
Column 21 of 10980
Column 22 of 10980
Column 23 of 10980
Column 24 of 10980
Column 25 of 10980
Column 26 of 10980
Column 27 of 10980
Column 28 of 10980
Column 29 of 10980
Column 30 of 10980
Column 31 of 10980
Column 32 of 10980
Column 33 of 10980
Column 34 of 10980
Column 35 of 10980
Column 36 of 10980
Column 37 of 10980
Column 38 of 10980
Column 39 of 10980
Column 40 of 10980
Column 41 of 10980
Column 42 of 10980
Column 43 of 10980

Column 411 of 10980
Column 412 of 10980
Column 413 of 10980
Column 414 of 10980
Column 415 of 10980
Column 416 of 10980
Column 417 of 10980
Column 418 of 10980
Column 419 of 10980
Column 420 of 10980
Column 421 of 10980
Column 422 of 10980
Column 423 of 10980
Column 424 of 10980
Column 425 of 10980
Column 426 of 10980
Column 427 of 10980
Column 428 of 10980
Column 429 of 10980
Column 430 of 10980
Column 431 of 10980
Column 432 of 10980
Column 433 of 10980
Column 434 of 10980
Column 435 of 10980
Column 436 of 10980
Column 437 of 10980
Column 438 of 10980
Column 439 of 10980
Column 440 of 10980
Column 441 of 10980
Column 442 of 10980
Column 443 of 10980
Column 444 of 10980
Column 445 of 10980
Column 446 of 10980
Column 447 of 10980
Column 448 of 10980
Column 449 of 10980
Column 450 of 10980
Column 451 of 10980
Column 452 of 10980
Column 453 of 10980
Column 454 of 10980
Column 455 of 10980
Column 456 of 10980
Column 457 of 10980
Column 458 of 10980
Column 459 of 10980
Column 460 of 10980


Column 821 of 10980
Column 822 of 10980
Column 823 of 10980
Column 824 of 10980
Column 825 of 10980
Column 826 of 10980
Column 827 of 10980
Column 828 of 10980
Column 829 of 10980
Column 830 of 10980
Column 831 of 10980
Column 832 of 10980
Column 833 of 10980
Column 834 of 10980
Column 835 of 10980
Column 836 of 10980
Column 837 of 10980
Column 838 of 10980
Column 839 of 10980
Column 840 of 10980
Column 841 of 10980
Column 842 of 10980
Column 843 of 10980
Column 844 of 10980
Column 845 of 10980
Column 846 of 10980
Column 847 of 10980
Column 848 of 10980
Column 849 of 10980
Column 850 of 10980
Column 851 of 10980
Column 852 of 10980
Column 853 of 10980
Column 854 of 10980
Column 855 of 10980
Column 856 of 10980
Column 857 of 10980
Column 858 of 10980
Column 859 of 10980
Column 860 of 10980
Column 861 of 10980
Column 862 of 10980
Column 863 of 10980
Column 864 of 10980
Column 865 of 10980
Column 866 of 10980
Column 867 of 10980
Column 868 of 10980
Column 869 of 10980
Column 870 of 10980


Column 1221 of 10980
Column 1222 of 10980
Column 1223 of 10980
Column 1224 of 10980
Column 1225 of 10980
Column 1226 of 10980
Column 1227 of 10980
Column 1228 of 10980
Column 1229 of 10980
Column 1230 of 10980
Column 1231 of 10980
Column 1232 of 10980
Column 1233 of 10980
Column 1234 of 10980
Column 1235 of 10980
Column 1236 of 10980
Column 1237 of 10980
Column 1238 of 10980
Column 1239 of 10980
Column 1240 of 10980
Column 1241 of 10980
Column 1242 of 10980
Column 1243 of 10980
Column 1244 of 10980
Column 1245 of 10980
Column 1246 of 10980
Column 1247 of 10980
Column 1248 of 10980
Column 1249 of 10980
Column 1250 of 10980
Column 1251 of 10980
Column 1252 of 10980
Column 1253 of 10980
Column 1254 of 10980
Column 1255 of 10980
Column 1256 of 10980
Column 1257 of 10980
Column 1258 of 10980
Column 1259 of 10980
Column 1260 of 10980
Column 1261 of 10980
Column 1262 of 10980
Column 1263 of 10980
Column 1264 of 10980
Column 1265 of 10980
Column 1266 of 10980
Column 1267 of 10980
Column 1268 o

Column 1617 of 10980
Column 1618 of 10980
Column 1619 of 10980
Column 1620 of 10980
Column 1621 of 10980
Column 1622 of 10980
Column 1623 of 10980
Column 1624 of 10980
Column 1625 of 10980
Column 1626 of 10980
Column 1627 of 10980
Column 1628 of 10980
Column 1629 of 10980
Column 1630 of 10980
Column 1631 of 10980
Column 1632 of 10980
Column 1633 of 10980
Column 1634 of 10980
Column 1635 of 10980
Column 1636 of 10980
Column 1637 of 10980
Column 1638 of 10980
Column 1639 of 10980
Column 1640 of 10980
Column 1641 of 10980
Column 1642 of 10980
Column 1643 of 10980
Column 1644 of 10980
Column 1645 of 10980
Column 1646 of 10980
Column 1647 of 10980
Column 1648 of 10980
Column 1649 of 10980
Column 1650 of 10980
Column 1651 of 10980
Column 1652 of 10980
Column 1653 of 10980
Column 1654 of 10980
Column 1655 of 10980
Column 1656 of 10980
Column 1657 of 10980
Column 1658 of 10980
Column 1659 of 10980
Column 1660 of 10980
Column 1661 of 10980
Column 1662 of 10980
Column 1663 of 10980
Column 1664 o

...

Column 4878 of 10980
Column 4879 of 10980
Column 4880 of 10980
Column 4881 of 10980
Column 4882 of 10980
Column 4883 of 10980
Column 4884 of 10980
Column 4885 of 10980
Column 4886 of 10980
Column 4887 of 10980
Column 4888 of 10980
Column 4889 of 10980
Column 4890 of 10980
Column 4891 of 10980
Column 4892 of 10980
Column 4893 of 10980
Column 4894 of 10980
Column 4895 of 10980
Column 4896 of 10980
Column 4897 of 10980
Column 4898 of 10980
Column 4899 of 10980
Column 4900 of 10980
Column 4901 of 10980
Column 4902 of 10980
Column 4903 of 10980
Column 4904 of 10980
Column 4905 of 10980
Column 4906 of 10980
Column 4907 of 10980
Column 4908 of 10980
Column 4909 of 10980
Column 4910 of 10980
Column 4911 of 10980
Column 4912 of 10980
Column 4913 of 10980
Column 4914 of 10980
Column 4915 of 10980
Column 4916 of 10980
Column 4917 of 10980
Column 4918 of 10980
Column 4919 of 10980
Column 4920 of 10980
Column 4921 of 10980
Column 4922 of 10980
Column 4923 of 10980
Column 4924 of 10980
Column 4925 o

Column 5269 of 10980
Column 5270 of 10980
Column 5271 of 10980
Column 5272 of 10980
Column 5273 of 10980
Column 5274 of 10980
Column 5275 of 10980
Column 5276 of 10980
Column 5277 of 10980
Column 5278 of 10980
Column 5279 of 10980
Column 5280 of 10980
Column 5281 of 10980
Column 5282 of 10980
Column 5283 of 10980
Column 5284 of 10980
Column 5285 of 10980
Column 5286 of 10980
Column 5287 of 10980
Column 5288 of 10980
Column 5289 of 10980
Column 5290 of 10980
Column 5291 of 10980
Column 5292 of 10980
Column 5293 of 10980
Column 5294 of 10980
Column 5295 of 10980
Column 5296 of 10980
Column 5297 of 10980
Column 5298 of 10980
Column 5299 of 10980
Column 5300 of 10980
Column 5301 of 10980
Column 5302 of 10980
Column 5303 of 10980
Column 5304 of 10980
Column 5305 of 10980
Column 5306 of 10980
Column 5307 of 10980
Column 5308 of 10980
Column 5309 of 10980
Column 5310 of 10980
Column 5311 of 10980
Column 5312 of 10980
Column 5313 of 10980
Column 5314 of 10980
Column 5315 of 10980
Column 5316 o

Column 5663 of 10980
Column 5664 of 10980
Column 5665 of 10980
Column 5666 of 10980
Column 5667 of 10980
Column 5668 of 10980
Column 5669 of 10980
Column 5670 of 10980
Column 5671 of 10980
Column 5672 of 10980
Column 5673 of 10980
Column 5674 of 10980
Column 5675 of 10980
Column 5676 of 10980
Column 5677 of 10980
Column 5678 of 10980
Column 5679 of 10980
Column 5680 of 10980
Column 5681 of 10980
Column 5682 of 10980
Column 5683 of 10980
Column 5684 of 10980
Column 5685 of 10980
Column 5686 of 10980
Column 5687 of 10980
Column 5688 of 10980
Column 5689 of 10980
Column 5690 of 10980
Column 5691 of 10980
Column 5692 of 10980
Column 5693 of 10980
Column 5694 of 10980
Column 5695 of 10980
Column 5696 of 10980
Column 5697 of 10980
Column 5698 of 10980
Column 5699 of 10980
Column 5700 of 10980
Column 5701 of 10980
Column 5702 of 10980
Column 5703 of 10980
Column 5704 of 10980
Column 5705 of 10980
Column 5706 of 10980
Column 5707 of 10980
Column 5708 of 10980
Column 5709 of 10980
Column 5710 o

Column 6054 of 10980
Column 6055 of 10980
Column 6056 of 10980
Column 6057 of 10980
Column 6058 of 10980
Column 6059 of 10980
Column 6060 of 10980
Column 6061 of 10980
Column 6062 of 10980
Column 6063 of 10980
Column 6064 of 10980
Column 6065 of 10980
Column 6066 of 10980
Column 6067 of 10980
Column 6068 of 10980
Column 6069 of 10980
Column 6070 of 10980
Column 6071 of 10980
Column 6072 of 10980
Column 6073 of 10980
Column 6074 of 10980
Column 6075 of 10980
Column 6076 of 10980
Column 6077 of 10980
Column 6078 of 10980
Column 6079 of 10980
Column 6080 of 10980
Column 6081 of 10980
Column 6082 of 10980
Column 6083 of 10980
Column 6084 of 10980
Column 6085 of 10980
Column 6086 of 10980
Column 6087 of 10980
Column 6088 of 10980
Column 6089 of 10980
Column 6090 of 10980
Column 6091 of 10980
Column 6092 of 10980
Column 6093 of 10980
Column 6094 of 10980
Column 6095 of 10980
Column 6096 of 10980
Column 6097 of 10980
Column 6098 of 10980
Column 6099 of 10980
Column 6100 of 10980
Column 6101 o

Column 6451 of 10980
Column 6452 of 10980
Column 6453 of 10980
Column 6454 of 10980
Column 6455 of 10980
Column 6456 of 10980
Column 6457 of 10980
Column 6458 of 10980
Column 6459 of 10980
Column 6460 of 10980
Column 6461 of 10980
Column 6462 of 10980
Column 6463 of 10980
Column 6464 of 10980
Column 6465 of 10980
Column 6466 of 10980
Column 6467 of 10980
Column 6468 of 10980
Column 6469 of 10980
Column 6470 of 10980
Column 6471 of 10980
Column 6472 of 10980
Column 6473 of 10980
Column 6474 of 10980
Column 6475 of 10980
Column 6476 of 10980
Column 6477 of 10980
Column 6478 of 10980
Column 6479 of 10980
Column 6480 of 10980
Column 6481 of 10980
Column 6482 of 10980
Column 6483 of 10980
Column 6484 of 10980
Column 6485 of 10980


In [ ]:
# GEDI array intersect with Sentinel-2 coordinates